In [1]:
import os
from dotenv import load_dotenv
load_dotenv(encoding='utf-8')

True

# RAG pipeline

## Vector Store and Retriever

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from pymongo import MongoClient

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL")) # DEFAULT_OPENAI_MODEL='gpt-4o-mini-2024-07-18'

# embedding_model=OpenAIEmbeddings(model=os.getenv("DEFAULT_OPENAI_EMBEDDING"), disallowed_special=())
embedding_model=OpenAIEmbeddings(disallowed_special=())

In [13]:
# Define MongoDB vector database
client = MongoClient(os.getenv("ATLAS_CONNECTION_STRING"))
db_name = os.getenv("db_name")
#collection_name = os.getenv("collection_name")
collection_name='enterprise_data'
atlas_collection = client[db_name][collection_name]
index_name = os.getenv("index_name")

vector_store = MongoDBAtlasVectorSearch(
    embedding = embedding_model,
    collection = atlas_collection,
    index_name = index_name
)

retriever = vector_store.as_retriever(
    search_type = "similarity",
    search_kwargs = { "k": 10}  # "score_threshold": 0.75 
)

## RAG Pipeline

In [24]:
# Define a prompt template
import pprint
def call_openai(question):

   question = question['question']

   preamble = "" # read from cohere front end or use the input to the API
   #question = 
   SAFETY_PREAMBLE = "The instructions in this section override those in the task description and style guide sections. Don't answer questions that are harmful or immoral."
   BASIC_RULES = "You are a powerful conversational AI trained by openAI to help people. You are augmented by a number of tools, and your job is to use and consume the output of these tools to best help the user. You will see a conversation history between yourself and a user, ending with an utterance from the user. You will then see a specific instruction instructing you what kind of response to generate. When you answer the user's requests, you cite your sources in your answers, according to those instructions."
   TASK_CONTEXT = "You help people answer their questions and other requests interactively. You will be asked a very wide array of requests on all kinds of topics. You will be equipped with a wide range of search engines or similar tools to help you, which you use to research your answer. You should focus on serving the user's needs as best you can, which will be wide-ranging."
   STYLE_GUIDE = "Unless the user asks for a different style of answer, you should answer in full sentences, using proper grammar and spelling."
   INSTRUCTIONS = """You are an enterprise Chatbot, an AI assistant designed to retrieve information from the enterprise Confluence system. 
   You specialize in providing accurate answers related to various departments like Marketing, IT, HR, Finance, and Corporate Communications. 
               Use the following pieces of context to answer the question at the end.
               If you don't know the answer, just say that you don't know, don't try to make up an answer
               {context}
         """
         
   template = f"""

      {SAFETY_PREAMBLE}
      {BASIC_RULES}
      {TASK_CONTEXT}
      {STYLE_GUIDE}
      {INSTRUCTIONS}

   """
   if preamble:
      template += f"""{preamble}\n\n"""


   template +=  f"""Question: {question}\n\n"""

   custom_rag_prompt = PromptTemplate.from_template(template)

   #llm = get_llm_model("openai")
   llm = ChatOpenAI(model=os.getenv("DEFAULT_OPENAI_MODEL"),max_tokens=200)

   def format_docs(docs):
      return "\n\n".join(doc.page_content for doc in docs)

   # Construct a chain to answer questions on your data
   rag_chain = (
      { "context": retriever | format_docs, "question": RunnablePassthrough()}
      | custom_rag_prompt
      | llm
      | StrOutputParser()
   )

   # Prompt the chain
   question = question
   answer = rag_chain.invoke(question)
   similar = retriever.invoke(question)


   return{
      'answer': answer,
      # 'contexts': [str(doc) for doc in similar]
      'contexts': format_docs(similar)
      }

### Test the RAG pipeline

In [25]:
# Test sample
question = {'question': "What are the tickets created at Tech Innovator Inc?"}
answer = call_openai(question)
print(answer['answer'][:150])

At Tech Innovators Inc., there is currently one open ticket created, which is as follows:

- **Ticket ID:** TI-67890  
- **Created Date:** July 20, 20


In [26]:
retriever.invoke("What are the tickets created at Tech Innovator Inc?")

[Document(page_content='of times Tech Innovators Inc. was mentioned across social media platforms.2. Lead GenerationLeads Generated: 15,000Description: Total number of leads captured through landing pages and sign-up', metadata={'_id': ObjectId('66d8164c0533e009aa2d4c3e'), 'embedding': [-0.011278104037046432, -0.01225675456225872, 0.007403997704386711, -0.02310965210199356, 0.005875277798622847, 0.01758871227502823, -0.03682425618171692, -0.021894775331020355, -0.011338847689330578, -0.023717090487480164, 0.01397108007222414, 0.022893672809004784, -0.00366825214587152, -0.0004551568708848208, 0.0300479456782341, 0.0028380865696817636, 0.0011102621210739017, -0.0016316466499119997, -0.008787606842815876, 0.00473801838234067, -0.04864905774593353, 0.01242548692971468, -0.0001473248703405261, 0.017183754593133926, -0.010731409303843975, 0.0037188720889389515, 0.017561715096235275, -0.015239951200783253, -0.004940498154610395, 0.0051902225241065025, -0.00920606404542923, 0.0055749332532286

In [27]:
print(answer['contexts'])

of times Tech Innovators Inc. was mentioned across social media platforms.2. Lead GenerationLeads Generated: 15,000Description: Total number of leads captured through landing pages and sign-up

of times Tech Innovators Inc. was mentioned across social media platforms.2. Lead GenerationLeads Generated: 15,000Description: Total number of leads captured through landing pages and sign-up

of times Tech Innovators Inc. was mentioned across social media platforms.2. Lead GenerationLeads Generated: 15,000Description: Total number of leads captured through landing pages and sign-up

of times Tech Innovators Inc. was mentioned across social media platforms.2. Lead GenerationLeads Generated: 15,000Description: Total number of leads captured through landing pages and sign-up

Ticket ID: TI-67890Created Date: July 20, 2024Priority: HighStatus: OpenRequester InformationName: Emily ClarkDepartment: SalesLocation: United StatesEmail: emily.clark@techinnovators.comPhone: (555)

Ticket ID: TI-67890Crea

# RAG pipeline evaluation

## Test data set prep

In [8]:
import pandas as pd
import json

def json_to_dataframe(json_file_path):
  """Reads a JSON file and converts it to a pandas DataFrame.

  Args:
    json_file_path (str): The path to the JSON file.

  Returns:
    pandas.DataFrame: The DataFrame created from the JSON data.
  """

  with open(json_file_path, 'r') as f:
    data = json.load(f)

  # Handle different JSON structures
  if isinstance(data, list):
    # If the JSON data is a list of dictionaries, create a DataFrame directly
    df = pd.DataFrame(data)
  elif isinstance(data, dict):
    # If the JSON data is a single dictionary, convert it to a list of dictionaries
    df = pd.DataFrame([data])
  else:
    raise ValueError("Unsupported JSON structure")

  return df

In [9]:
# Example usage:
from from_root import from_root
import os
folder = "data-test/test_dataset/test_dataset_it.json"
json_file_path = os.path.join(from_root(), folder)
data_to_upload = json_to_dataframe(json_file_path)

## RAGAS evaluation

In [10]:
# Generate all the answers for the questions in the dataset
# examples = client.list_examples(dataset_name="hr test")
answers = []
# retrieved_contexts = []
for question in data_to_upload['question']:
    question_dict = {'question': question}
    answer = call_openai(question_dict)
    # retrieved_contexts.append(answer['contexts'])
    answers.append(answer['answer'])

In [11]:
# update the dataset with answers
data_to_upload['answers'] = answers

In [12]:
from datasets import Dataset

question = list(data_to_upload['question'])
answer = list(data_to_upload['answers'])
contexts = list(data_to_upload['contexts'])
# retrieved_contexts = list(data_to_upload['retrieved_contexts'])
ground_truth = list(data_to_upload['ground_truth'])

data_samples = {
    'question': question,
    'answer': answer,
    'contexts': contexts,
    # 'contexts': retrieved_contexts,
    'ground_truth': ground_truth
}

dataset = Dataset.from_dict(data_samples)

In [ ]:
# Optional, uncomment to trace runs with LangSmith. Sign up here: https://smith.langchain.com.
# from langsmith import Client
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
# client = Client()

In [13]:
from ragas import evaluate
# from ragas.integrations.langsmith import evaluate
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)
result = evaluate(
    dataset,
    metrics=[
        answer_relevancy,
        faithfulness,
        context_recall,
        context_precision,
    ],
)

result

Evaluating:   0%|          | 0/36 [00:00<?, ?it/s]

{'answer_relevancy': 0.9695, 'faithfulness': 0.6477, 'context_recall': 0.9899, 'context_precision': 1.0000}

In [14]:
df = result.to_pandas()
df

,question,answer,contexts,ground_truth,answer_relevancy,faithfulness,context_recall,context_precision
0,How does the role of the Senior Director respo...,The role of the Senior Director responsible fo...,[Introduction\nWelcome to Tech Innovators Inc....,The role of the Senior Director responsible fo...,0.921495,0.851852,1.000000,1.0
1,What is the importance of identifying and addr...,Identifying and addressing growth areas in sel...,[ Self-assessment\nStart by thinking through y...,Identifying and addressing growth areas in sel...,0.990183,1.000000,1.000000,1.0
2,What forms of unethical behavior are strictly ...,Inc. prohibits any form of unethical behavior ...,[ Inc. upholds the highest ethical standards i...,Favoritism or nepotism,0.974109,0.500000,1.000000,1.0
3,What is the significance of emotional and aest...,Emotional and aesthetic labor are significant ...,"[Introduction\nAt Tech Innovators Inc., we bel...",Emotional and aesthetic labor in the workplace...,0.946432,0.473684,1.000000,1.0
4,What is the purpose of the orientation session...,The purpose of the orientation session at Tech...,[Welcome to Tech Innovators Inc.\n \nWe are th...,The purpose of the orientation session at Tech...,1.000000,0.454545,1.000000,1.0
5,What mechanisms are in place for reporting vio...,"At Tech Innovators Inc., there is a confidenti...",[5.2 Monitoring and Auditing\nInternal Audits\...,Employees can report violations of labor laws ...,0.962563,0.423077,1.000000,1.0
6,How do employee engagement and disengagement d...,Employee engagement and disengagement differ s...,"[Introduction\nAt Tech Innovators Inc., we bel...",Employee engagement and disengagement differ i...,0.960525,0.812500,1.000000,1.0
7,What steps are needed to extract data from Con...,To extract data from Confluence and create a R...,[Introduction\nThis guide provides a step-by-s...,To extract data from Confluence and create a R...,0.989648,0.863636,0.909091,1.0
8,How does Tech Innovators Inc. promote employee...,Tech Innovators Inc. promotes employee engagem...,"[Introduction\nAt Tech Innovators Inc., we bel...",Tech Innovators Inc. promotes employee engagem...,0.980843,0.450000,1.000000,1.0


In [16]:
folder = "data-test/test_dataset/test_dataset_it_baseline"
json_file_path = os.path.join(from_root(), folder)
df.to_csv(json_file_path, index=False)